# Exploratory Data Analysis (EDA) - USC Speech MRI Dataset

**Project Sullivan - Milestone M1**

**Date:** 2025-11-25

**Researcher:** [Your Name]

**Objective:** Understand the structure and characteristics of the USC-TIMIT Speech MRI dataset to inform preprocessing strategies for Phase 1.

---

## Tasks

- [ ] Verify data download and extraction
- [ ] Analyze MRI frame properties (resolution, fps, format)
- [ ] Analyze audio properties (sample rate, duration, format)
- [ ] Check audio-MRI synchronization
- [ ] Visualize sample MRI frames and spectrograms
- [ ] Identify data quality issues (noise, missing frames)
- [ ] Document findings in `docs/data_statistics.md`

## 1. Setup & Imports

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import json

# Audio processing
import librosa
import librosa.display
import soundfile as sf

# Image processing
import cv2
from PIL import Image

# Utils
from tqdm import tqdm
import pandas as pd

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Project root
PROJECT_ROOT = Path('/home/midori/Develop/Project_Sullivan')
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'

print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Directory: {DATA_RAW}")
print(f"Python Version: {sys.version}")

## 2. Data Discovery

In [ ]:
# List all directories in data/raw
print("Contents of data/raw:")
for item in DATA_RAW.iterdir():
    if item.is_dir():
        size = sum(f.stat().st_size for f in item.rglob('*') if f.is_file())
        print(f"  - {item.name}: {size / 1e6:.2f} MB")

In [ ]:
# Find MRI files (update path based on actual data structure)
mri_files = sorted(glob(str(DATA_RAW / '**' / '*.png'), recursive=True))
mri_files += sorted(glob(str(DATA_RAW / '**' / '*.jpg'), recursive=True))
mri_files += sorted(glob(str(DATA_RAW / '**' / '*.mp4'), recursive=True))

print(f"Total MRI-related files found: {len(mri_files)}")
if mri_files:
    print(f"\nFirst 5 files:")
    for f in mri_files[:5]:
        print(f"  {f}")
else:
    print("\n⚠️ No MRI files found! Please download data from figshare.")
    print("See: docs/DATA_DOWNLOAD_GUIDE.md")

In [ ]:
# Find audio files
audio_files = sorted(glob(str(DATA_RAW / '**' / '*.wav'), recursive=True))
audio_files += sorted(glob(str(DATA_RAW / '**' / '*.mp3'), recursive=True))

print(f"Total audio files found: {len(audio_files)}")
if audio_files:
    print(f"\nFirst 5 files:")
    for f in audio_files[:5]:
        print(f"  {f}")
else:
    print("\n⚠️ No audio files found!")

## 3. MRI Frame Analysis

**Note:** This section assumes you have downloaded MRI frames. If not, skip to Section 7.

In [ ]:
if mri_files and mri_files[0].endswith('.png'):
    # Load sample MRI frame
    sample_mri_path = mri_files[0]
    sample_mri = cv2.imread(sample_mri_path, cv2.IMREAD_GRAYSCALE)
    
    print(f"Sample MRI: {sample_mri_path}")
    print(f"  Shape: {sample_mri.shape}")
    print(f"  Dtype: {sample_mri.dtype}")
    print(f"  Value range: [{sample_mri.min()}, {sample_mri.max()}]")
    print(f"  Mean: {sample_mri.mean():.2f}")
    print(f"  Std: {sample_mri.std():.2f}")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(sample_mri, cmap='gray')
    axes[0].set_title('Sample MRI Frame')
    axes[0].axis('off')
    
    axes[1].hist(sample_mri.ravel(), bins=50, color='skyblue', edgecolor='black')
    axes[1].set_title('Pixel Intensity Distribution')
    axes[1].set_xlabel('Pixel Value')
    axes[1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'results' / 'eda_mri_sample.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("⚠️ Skipping MRI analysis - no PNG/JPG frames found")

## 4. Audio Analysis

In [ ]:
if audio_files:
    # Load sample audio
    sample_audio_path = audio_files[0]
    audio, sr = librosa.load(sample_audio_path, sr=None)
    
    print(f"Sample Audio: {sample_audio_path}")
    print(f"  Sample rate: {sr} Hz")
    print(f"  Duration: {len(audio) / sr:.2f} seconds")
    print(f"  Length: {len(audio)} samples")
    print(f"  Value range: [{audio.min():.4f}, {audio.max():.4f}]")
    print(f"  RMS: {np.sqrt(np.mean(audio**2)):.4f}")
else:
    print("⚠️ Skipping audio analysis - no audio files found")

In [ ]:
if audio_files:
    # Visualize waveform and spectrogram
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Waveform
    librosa.display.waveshow(audio, sr=sr, ax=axes[0])
    axes[0].set_title('Audio Waveform')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    
    # Spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    img = librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz', ax=axes[1])
    axes[1].set_title('Spectrogram')
    fig.colorbar(img, ax=axes[1], format='%+2.0f dB')
    
    # Mel Spectrogram
    S = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=80)
    S_db = librosa.power_to_db(S, ref=np.max)
    img2 = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='mel', ax=axes[2])
    axes[2].set_title('Mel Spectrogram')
    fig.colorbar(img2, ax=axes[2], format='%+2.0f dB')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'results' / 'eda_audio_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

## 5. Audio-MRI Synchronization Check

**Goal:** Verify that audio and MRI frames are temporally aligned.

In [ ]:
# This section requires knowing the correspondence between audio and MRI files
# Update based on actual dataset structure

# Example:
# - MRI frame rate: 50 fps
# - Audio sample rate: 20000 Hz
# - Expected samples per frame: 20000 / 50 = 400 samples

if mri_files and audio_files:
    # Placeholder for synchronization analysis
    print("⚠️ TODO: Implement synchronization check based on metadata")
    print("\nExpected:")
    print("  - Frame timestamps from MRI metadata")
    print("  - Audio timestamps from sync signal or metadata")
    print("  - Cross-correlation for precise alignment")
else:
    print("⚠️ Cannot check synchronization without both MRI and audio data")

## 6. Dataset Statistics Summary

In [ ]:
# Compile statistics
stats = {
    'dataset': 'USC-TIMIT Speech MRI',
    'data_downloaded': len(mri_files) > 0 or len(audio_files) > 0,
    'mri_files_count': len(mri_files),
    'audio_files_count': len(audio_files),
}

if mri_files and mri_files[0].endswith('.png'):
    stats['mri_resolution'] = f"{sample_mri.shape[1]}x{sample_mri.shape[0]}"
    stats['mri_dtype'] = str(sample_mri.dtype)
    
if audio_files:
    stats['audio_sample_rate'] = sr
    stats['audio_duration_sec'] = len(audio) / sr

# Display
print("\n" + "="*50)
print("DATASET STATISTICS SUMMARY")
print("="*50)
for key, value in stats.items():
    print(f"{key:25s}: {value}")
print("="*50)

# Save to JSON
stats_path = PROJECT_ROOT / 'docs' / 'data_statistics.json'
with open(stats_path, 'w') as f:
    json.dump(stats, f, indent=2)
print(f"\nStatistics saved to: {stats_path}")

## 7. Next Steps & Action Items

### If Data is NOT Downloaded:

- [ ] **Download data from figshare** (see `docs/DATA_DOWNLOAD_GUIDE.md`)
- [ ] Start with 1-2 subjects for testing
- [ ] Re-run this notebook after download

### If Data IS Downloaded:

- [ ] **Document findings** in `docs/data_statistics.md`
- [ ] **Identify preprocessing needs:**
  - MRI noise reduction required?
  - Audio filtering needed?
  - Synchronization method?
- [ ] **Proceed to Phase 1 - Step 2:** Denoising & Alignment
- [ ] **Create preprocessing scripts** in `src/preprocessing/`

### Research Log Entry

- [ ] Update research log with EDA findings
- [ ] Note any data quality issues
- [ ] Plan next week's tasks

---

**Experiment ID:** EXP-20251125-01-EDA

**Status:** [In Progress / Completed]

**Notes:** 

---